# Development Environment
Google Colab
- Platfrom : Linux-5.10.147+-x86_64-with-glibc2.29
- OS:Ubuntu 20.04.5 LTS
- CPU : Intel(R) Xeon(R) CPU @ 2.20GHz

# Python Library Version
- Python 3.8.10
- pandas 1.3.5
- numpy 1.22.4
- sklearn 1.2.1
- xgboost 1.7.4
- lightgbm 2.2.3


In [ ]:
!python -V

Python 3.8.10


In [ ]:
#-*- coding:utf-8 -*-

import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
train_df = pd.read_csv('/data/train.csv') # train data load
test_df = pd.read_csv('/data/test.csv') # test data load

In [ ]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality']) # feature로 사용하지 않을 column 제거
train_y = train_df['Y_Class'] # target

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP']) # feature로 사용하지 않을 column 제거

In [ ]:
# 결측치 0으로 대체
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE'] # numerical하지 않은 column

# label encoding
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [ ]:
# library download
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# version 확인
import xgboost
xgboost.__version__

'1.7.4'

In [ ]:
from xgboost import XGBClassifier # library import

xgb_clf = XGBClassifier(random_state=37).fit(train_x, train_y) # training
preds = xgb_clf.predict(test_x) # test data prediction

In [ ]:
preds

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
submit = pd.read_csv('/data/sample_submission.csv') # 제출 template Load

In [ ]:
submit['Y_Class'] = preds # 제출 데이터 입력

In [ ]:
submit.to_csv('/data/XGB.csv', index=False) # 제출용 파일 

In [ ]:
xgb_clf.save_model("/data/XGB_model.json") # 모델 저장